# Tutorial

This example provides a simply query tutorial for InfluxDB Cloud Dedicated. 

In [10]:
import influxdb_client_3 as InfluxDBClient3
import plotly.io as pio
import plotly.express as px
import pandas as pd
import pyarrow.parquet as pq
from pyarrow import csv
pio.renderers.default = "vscode"

Next we provide our connection credentials to the InfluxDB 3.0 instance. 
`token` - scoped to the bucket you wish to query from.
`host` - the host of your cloud dedicated instance.
`client` - host wrapped in the flightclient class. 

In [11]:
token = ''
host = 'eu-central-1-1.aws.cloud2.influxdata.com'    
org = 'Jay-IOx'
db = 'factory'

client = InfluxDBClient3.InfluxDBClient3(token=token,
                         host=host,
                         org=org,
                         database=db) 
    
    


In [12]:
query = "SELECT vibration FROM machine_data WHERE time >= now() - 1h GROUP BY machineID"
table = client.query(query=query, language="influxql")
display(table)

pyarrow.Table
iox::measurement: string not null
time: timestamp[ns] not null
machineID: string
vibration: double
----
iox::measurement: [["machine_data","machine_data","machine_data","machine_data","machine_data",...,"machine_data","machine_data","machine_data","machine_data","machine_data"]]
time: [[2023-06-21 12:07:54.799332500,2023-06-21 12:07:55.031291809,2023-06-21 12:07:56.031963473,2023-06-21 12:07:57.033159195,2023-06-21 12:07:58.033690069,...,2023-06-21 12:43:44.938057536,2023-06-21 12:43:45.938740018,2023-06-21 12:43:46.939135645,2023-06-21 12:43:47.940752925,2023-06-21 12:43:48.941473556]]
machineID: [["machine1","machine1","machine1","machine1","machine1",...,"machine3","machine3","machine3","machine3","machine3"]]
vibration: [[71,58,60,73,74,...,85,81,87,80,80]]

In [13]:
df = table.to_pandas()
display(df)

,iox::measurement,time,machineID,vibration
0,machine_data,2023-06-21 12:07:54.799332500,machine1,71.0
1,machine_data,2023-06-21 12:07:55.031291809,machine1,58.0
2,machine_data,2023-06-21 12:07:56.031963473,machine1,60.0
3,machine_data,2023-06-21 12:07:57.033159195,machine1,73.0
4,machine_data,2023-06-21 12:07:58.033690069,machine1,74.0
...,...,...,...,...
6457,machine_data,2023-06-21 12:43:44.938057536,machine3,85.0
6458,machine_data,2023-06-21 12:43:45.938740018,machine3,81.0
6459,machine_data,2023-06-21 12:43:46.939135645,machine3,87.0
6460,machine_data,2023-06-21 12:43:47.940752925,machine3,80.0


In [14]:
fig = px.line(df, x="time", y=["vibration"], color="machineID", title='raw vibration')
fig.show()

In [15]:
aggregation = table.group_by("machineID").aggregate([("vibration", "mean"), 
                                                     ("vibration", "max"), 
                                                     ("vibration", "min") ]).to_pandas()
display(aggregation)

,vibration_mean,vibration_max,vibration_min,machineID
0,84.845868,90.0,50.0,machine1
1,107.429434,598.0,55.0,machine2
2,84.722841,90.0,50.0,machine3


In [16]:
# Reshape Data Frame
aggregation=pd.melt(aggregation,id_vars=['machineID'],var_name='aggregation', value_name='value')
fig2 = px.bar(aggregation, x="machineID", y="value", color='aggregation', barmode= 'group', title='Vibration min,max,mean')
fig2.show()

In [17]:
# Write to Parquet
pq.write_table(table, 'machine_data.parquet', version='2.6')
csv.write_csv(table, "machine_data.csv")